In [0]:
!pip install guidedlda
!pip install pandas
!pip install gensim
!pip install os



     |████████████████████████████████| 2.1MB 2.8MB/s 
  Created wheel for guidedlda: filename=guidedlda-2.0.0.dev22-cp36-cp36m-linux_x86_64.whl size=2479786 sha256=32251a86823612945ec2eb68c8ba01b75030771c2b8cbbaa4d98c319bd245602
  Stored in directory: /root/.cache/pip/wheels/cf/cb/c4/95897cbe4e2e12bd01db3c5cb004c744d2cfd68ecc692ce39e
Successfully built guidedlda
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [0]:
import nltk
import nltk.stem as stemmer
import numpy as np

import pandas as pd
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.test.utils import get_tmpfile
import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords

In [0]:
import guidedlda

In [0]:

x=pd.read_csv("/content/science_new.csv")


In [0]:
X=[]
for i, row in x.iterrows():
        X.append(row['body_no_quote'])

In [0]:
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
stemmer = WordNetLemmatizer()
documents=[]
for sen in range(0, len(X)):
  
  document = re.sub(r'\W', ' ', str(X[sen]))
  document=document.lower()
  # remove all single characters
  document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
  
  # Remove single characters from the start
  document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
  
  # Substituting multiple spaces with single space
  document = re.sub(r'\s+', ' ', document, flags=re.I)
  
  # Removing prefixed 'b'
  document = re.sub(r'^b\s+', '', document)
  
  # Converting to Lowercase
  #document = document.lower()
  document=remove_stopwords(document)
  # Lemmatization
  document = document.split()

  document = [stemmer.lemmatize(word) for word in document]
  document = ' '.join(document)
  
  documents.append(document)              
        
'''
documents = list(dict.fromkeys(documents))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents).toarray()
'''
stop_words = set(stopwords.words('english'))
texts = [[text for text in doc.split()] for doc in documents]
for i in texts:

  for j in i:

    if j in stop_words:

      i.remove(j)
          
        
dictionary = corpora.Dictionary(texts)
print(dictionary)
tokenized_list = [simple_preprocess(doc) for doc in documents]

mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

matrix=[]
#Y=[]
#Y.extend(matrix)
vocab=tuple(mydict.token2id.keys())
for i in range(len(mycorpus)):

  word=[]
  count=[]
  temp=[]
  for tu in mycorpus[i]:
    
    word.append(tu[0])
    count.append(tu[1])
    #print(word,count)
  for j in range(0,len(vocab)):
    if j in word:
      temp.append(count[word.index(j)])
    else:
      temp.append(0)
  matrix.append(temp)

Dictionary(5224 unique tokens: ['blogspam', 'image', 'infographic', 'post', 'removed']...)


In [0]:
#vocab = guidedlda.datasets.load_vocab(guidedlda.datasets.NYT)

In [0]:
vocab=tuple(mydict.token2id.keys())

In [0]:
len(vocab)

4934

In [0]:
word2id = mydict.token2id

In [0]:
model = guidedlda.GuidedLDA(n_topics=4, n_iter=100, random_state=7, refresh=20)

In [0]:
X=np.array(matrix) 


In [0]:
X.shape

(3086, 4934)

In [0]:
type(X)

numpy.ndarray

In [0]:
model.fit(X)

INFO:guidedlda:n_documents: 3086
INFO:guidedlda:vocab_size: 4934
INFO:guidedlda:n_words: 35443
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -344574
INFO:guidedlda:<20> log likelihood: -253306
INFO:guidedlda:<40> log likelihood: -241671
INFO:guidedlda:<60> log likelihood: -238323
INFO:guidedlda:<80> log likelihood: -236420
INFO:guidedlda:<99> log likelihood: -235345


In [0]:
topic_word = model.topic_word_

In [0]:
n_top_words = 8

In [0]:
type(vocab)

tuple

In [0]:
len(vocab)

4934

In [0]:
len(mydict)

4934

In [0]:
for i, topic_dist in enumerate(topic_word):
>>>     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
>>>     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: science comment submission peer reviewed article rule removed
Topic 1: submission removed peer reviewed new research reference include
Topic 2: people study like thing work isn know mean
Topic 3: submission removed repost story submitted popular allowed sorry


In [0]:
len(vocab)

4934

In [0]:
"because" in vocab

False

In [0]:

seed_topic_list = [['removed', 'remove', 'banned', 'ban', 'warned', 'spam'],
               ['recommend', 'look', 'try'],
               ['rule','guideline','reason','removal',"dedicated","allowed","peer","reviewed" ]]

In [0]:
model = guidedlda.GuidedLDA(n_topics=4, n_iter=100, random_state=7, refresh=20)


In [0]:
seed_topics = {}

In [0]:
for t_id, st in enumerate(seed_topic_list):
>>>     for word in st:
>>>         seed_topics[word2id[word]] = t_id

In [0]:
model.fit(X, seed_topics=seed_topics, seed_confidence=0.2)

INFO:guidedlda:n_documents: 3086
INFO:guidedlda:vocab_size: 4934
INFO:guidedlda:n_words: 35443
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 100
INFO:guidedlda:<0> log likelihood: -343836
INFO:guidedlda:<20> log likelihood: -245400
INFO:guidedlda:<40> log likelihood: -241038
INFO:guidedlda:<60> log likelihood: -239368
INFO:guidedlda:<80> log likelihood: -238068
INFO:guidedlda:<99> log likelihood: -236812


In [0]:
n_top_words = 10
>>> topic_word = model.topic_word_
>>> for i, topic_dist in enumerate(topic_word):
>>>     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
>>>     print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: submission removed repost story submitted popular science comment link peer
Topic 1: thing like people think work isn know mean time study
Topic 2: submission removed peer reviewed new research reference include sorry hi
Topic 3: science comment http article people study www journal title com


In [0]:
doc_topic = model.transform(X)
>>> for i in range(9):
>>>     print("top topic: {} Document: {}".format(doc_topic[i].argmax(),
                                                  ', '.join(np.array(vocab)[list(reversed(X[i,:].argsort()))[0:5]])))

top topic: 0 Document: blogspam, image, infographic, post, removed
top topic: 2 Document: reviewed, new, submission, research, reference
top topic: 2 Document: reviewed, new, submission, research, reference
top topic: 2 Document: reviewed, new, submission, research, reference
top topic: 0 Document: removed, scientific, nature, submission, entitled
top topic: 2 Document: reviewed, new, submission, research, reference
top topic: 0 Document: removed, scientific, nature, submission, entitled
top topic: 2 Document: reviewed, new, submission, research, reference
top topic: 0 Document: reapproval, citation, comment, direct, submission


<class 'numpy.ndarray'>


[0.001233205868455609,
 0.00010469412787614679,
 0.9985271931341778,
 0.000134906869490618]

9.30783645220705e-05

In [0]:
df=pd.DataFrame(doc_topic)

In [0]:
df.applymap(float)

,0,1,2,3
0,0.635683,0.000172,0.363956,0.000188
1,0.000463,0.000013,0.999506,0.000018
2,0.000463,0.000013,0.999506,0.000018
3,0.000463,0.000013,0.999506,0.000018
4,0.989631,0.000130,0.009434,0.000806
...,...,...,...,...
3081,0.988293,0.011202,0.000014,0.000491
3082,0.000193,0.998922,0.000449,0.000437
3083,0.013890,0.963514,0.011267,0.011328
3084,0.002025,0.001096,0.003125,0.993754


In [0]:
re=pd.concat([x,df],axis=1)

In [0]:
re.to_csv("science_new_withlabels.csv")